# Import

In [ ]:
import os
import time

In [ ]:
import pandas as pd

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [ ]:
import transformers
import accelerate
print(transformers.__version__)
print(accelerate.__version__)

In [ ]:
print("CUDA 사용 가능:", torch.cuda.is_available())
print("GPU 개수:", torch.cuda.device_count())
print("사용 중인 GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU 사용 중")

# Data Load & Pre-processing

In [ ]:
path_train = "/workspace/Storage/hansoldeco3/Data/raw/train.csv"
path_test = "/workspace/Storage/hansoldeco3/Data/raw/test.csv"
base_pdf_dir = "/workspace/Storage/hansoldeco3/Data/raw/건설안전지침"
paths_pdf = os.listdir(base_pdf_dir)
pdf_paths = [os.path.join(base_pdf_dir, pdf_file) for pdf_file in paths_pdf]

In [ ]:
train = pd.read_csv(path_train, encoding = 'utf-8-sig')
test = pd.read_csv(path_test, encoding = 'utf-8-sig')

In [ ]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [ ]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [ ]:
# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

# Model import

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

print(model.hf_device_map)  # GPU 정상 할당 여부 확인

# Vector store 생성

In [ ]:
# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-nli"  # 임베딩 모델 선택
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [ ]:
# PDF 로드
pdf_docs = []
for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load()  # 문서 리스트 반환
    pdf_docs.extend(docs)

# 텍스트 chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(pdf_docs)

# PDF 벡터 스토어
vector_store_pdf = FAISS.from_documents(split_docs, embedding)

In [ ]:
split_docs[:5]

In [ ]:
# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]


# # 벡터 스토어에 문서 추가
# vector_store = FAISS.from_texts(train_documents, embedding)

# # Retriever 정의
# retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})


vector_store_train = FAISS.from_texts(train_documents, embedding)
vector_store_train.merge_from(vector_store_pdf)
retriever = vector_store_train.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# RAG chain 생성

In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,  # sampling 활성화
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=64,
)

prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

{context}

### 질문:
{question}

[/INST]

"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 단순 컨텍스트 결합 방식 사용
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}  # 커스텀 프롬프트 적용
)

# Inference

In [17]:
# 테스트 실행 및 결과 저장
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

for idx, row in combined_test_data.iterrows():

    t = time.time()
    print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")
    print(row['question'])
    # RAG 체인 호출 및 결과 생성
    prevention_result = qa_chain.invoke(
        row['question'],
        # llm_kwargs={"max_new_tokens": 50, "use_cache": True, "do_sample": False}
        )

    # 결과 저장
    result_text = prevention_result['result']
    test_results.append(result_text)
    print(f"iter {idx} 완료 : {time.time()-t}초")


    # ---- (추가) 참조된 문서(맥락) 출력 ----
    source_docs = prevention_result['source_documents']
    print("\n[참조된 문서/맥락]")
    for i, doc in enumerate(source_docs):
        # 필요 시 doc에 여러 정보가 있으므로, page_content만 간단히 출력
        print(f"--- 문서 {i+1} ---")
        print(doc.page_content, "...")  # 앞부분 500자만 잘라보기 등
    
    print(f"iter {idx} 완료 : {time.time()-t}초")
    break

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964

[샘플 1/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설기계'(중분류: '콘크리트펌프')와 관련된 사고가 발생했습니다. 작업 프로세스는 '타설작업'이며, 사고 원인은 '펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
iter 0 완료 : 28.810788869857788초

[참조된 문서/맥락]
--- 문서 1 ---
Q: 공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설기계'(중분류: '콘크리트펌프')와 관련된 사고가 발생했습니다. 작업 프로세스는 '타설작업'이며, 사고 원인은 '콘크리트 펌핑중 아웃리거가 흙바닥으로 밀리면서 흙바닥이 침하되어 펌프카가 기울어짐, 진동에 의한 아웃리거 변위'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
A: 펌프카 설치 위치 사전 검토와 아웃리거 변위 여부 점검을 통한 재발 방지 대책 마련. ...
--- 문서 2 ---
Q: 공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설기계'(중분류: '콘크리트펌프')와 관련된 사고가 발생했습니다. 작업 프로세스는 '타설작업'이며, 사고 원인은 '펌프카 붐대를 최대거리로 사용하면서 하중이 집중되어 아웃트리거 지반이 붕괴.'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
A: 관리적 대책으로 차량계 건설기계 작업계획서 작성 및 검토와 펌프

# Submission

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

In [ ]:
submission = pd.read_csv('/workspace/Storage/hansoldeco3/Data/sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')